In [1]:
import pandas as pd
df = pd.read_csv('train.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [2]:
df.groupby('Survived').mean()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
Survived,,,,,,
0,447.016393,2.531876,30.626179,0.553734,0.329690,22.117887
1,444.368421,1.950292,28.343690,0.473684,0.464912,48.395408


In [3]:
df.shape

(891, 12)

In [4]:
column = ['Pclass', 'Sex' ,'Age', 'Fare']
df2 = df[column]
df2

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500
...,...,...,...,...
886,2,male,27.0,13.0000
887,1,female,19.0,30.0000
888,3,female,NaN,23.4500
889,1,male,26.0,30.0000


In [5]:
df2.isnull().sum()

Pclass      0
Sex         0
Age       177
Fare        0
dtype: int64

In [6]:
df2.Age = df2.Age.fillna(df2.Age.mean())

/tmp/ipykernel_2484/2177684196.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.Age = df2.Age.fillna(df2.Age.mean())


In [7]:
df2.isna().sum()

Pclass    0
Sex       0
Age       0
Fare      0
dtype: int64

In [8]:
df2.Sex = pd.get_dummies(df2.Sex, drop_first=True)


/tmp/ipykernel_2484/4248454742.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.Sex = pd.get_dummies(df2.Sex, drop_first=True)


In [9]:
x = df2
y = df.Survived

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.7)

In [11]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


In [12]:
model_param = {
    'svm' : {
        'model' : SVC(gamma='auto'),
        'param' : {
            'C' : [1,10,20],
            'kernel' : ['rbf', 'linear']
        }
    },
    'random_forest' : {
        'model' : RandomForestClassifier(),
        'param' : {
            'n_estimators' : [1,5,10,50,100]
        }
    },
    'logidtic_regression' : {
        'model' : LogisticRegression(solver='liblinear', multi_class='auto'),
        'param' : {
            'C' : [1,5,10]
        }
    },
    'ggb' : {
        'model' : GaussianNB(),
        'param' : {

        },
    },
    'mgb' : {
        'model' : MultinomialNB(),
        'param' : {
            'alpha' : [0,0.5,0.7,0.2,1]
        }
    }
}

In [13]:
scores = []
for model_name, mp in model_param.items():
    clf = GridSearchCV(mp['model'],mp['param'], cv=5, return_train_score=False)
    clf.fit(x, y)
    scores.append({
        'model' : model_name,
        'best_score' : clf.best_score_,
        'best_param' : clf.best_params_
    })
scores

/home/opeyemi/anaconda3/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/home/opeyemi/anaconda3/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/home/opeyemi/anaconda3/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/home/opeyemi/anaconda3/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/home/opeyemi/anaconda3/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/home/opeyemi/anaconda3/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: al

[{'model': 'svm',
  'best_score': 0.7867365513778168,
  'best_param': {'C': 1, 'kernel': 'linear'}},
 {'model': 'random_forest',
  'best_score': 0.8193522063900571,
  'best_param': {'n_estimators': 10}},
 {'model': 'logidtic_regression',
  'best_score': 0.7901198920343984,
  'best_param': {'C': 1}},
 {'model': 'ggb', 'best_score': 0.7677421379699956, 'best_param': {}},
 {'model': 'mgb',
  'best_score': 0.6914694620551126,
  'best_param': {'alpha': 0}}]

In [26]:
score = pd.DataFrame(scores)
score

,model,best_score,best_param
0,svm,0.786737,"{'C': 1, 'kernel': 'linear'}"
1,random_forest,0.819352,{'n_estimators': 10}
2,logidtic_regression,0.790120,{'C': 1}
3,ggb,0.767742,{}
4,mgb,0.691469,{'alpha': 0}


In [28]:
real_model = RandomForestClassifier(n_estimators= 20)


In [29]:
real_model.fit(x,y)

RandomForestClassifier(n_estimators=20)

In [30]:
df_test = pd.read_csv('test.csv')
df_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [31]:
df2_test = df_test[column]
df2_test

,Pclass,Sex,Age,Fare
0,3,male,34.5,7.8292
1,3,female,47.0,7.0000
2,2,male,62.0,9.6875
3,3,male,27.0,8.6625
4,3,female,22.0,12.2875
...,...,...,...,...
413,3,male,NaN,8.0500
414,1,female,39.0,108.9000
415,3,male,38.5,7.2500
416,3,male,NaN,8.0500


In [32]:
df2_test.Sex = pd.get_dummies(df2_test.Sex, drop_first=True)
x_test = df2_test

/tmp/ipykernel_2484/4090273490.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_test.Sex = pd.get_dummies(df2_test.Sex, drop_first=True)


In [33]:
df2_test.isna().sum()

Pclass     0
Sex        0
Age       86
Fare       1
dtype: int64

In [34]:
df2_test.Age = df2_test.Age.fillna(df2_test.Age.mean())
df2_test.Fare = df2_test.Fare.fillna(df2_test.Fare.mean())
df2_test.isna().sum()

/tmp/ipykernel_2484/2410701328.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_test.Age = df2_test.Age.fillna(df2_test.Age.mean())
/tmp/ipykernel_2484/2410701328.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_test.Fare = df2_test.Fare.fillna(df2_test.Fare.mean())


Pclass    0
Sex       0
Age       0
Fare      0
dtype: int64

In [35]:
Survived = real_model.predict(x_test)


In [36]:
prediction = pd.DataFrame(df_test.PassengerId)
prediction['Survived'] = Survived
# prediction.to_csv('prediction.csv').set_

In [37]:
prediction_result = prediction.set_index('PassengerId')
prediction_result.to_csv('prediction.csv')

In [ ]:
df3 = pd.read_csv('prediction.csv')
df3.set_index('PassengerId',  inplace=True)

In [ ]:
df3.to_csv('prediction1.csv')

In [ ]:
df4 = pd.read_csv('prediction1.csv')
df4.shape

(418, 2)